In [2]:
# For Sam

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

loading the dataset

In [4]:
dataset = pd.read_csv("Train.csv")

Checking it out

In [5]:
dataset.head(2)

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,...,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,...,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993


In [6]:
riders = pd.read_csv('Riders.csv')
riders.head(2)

,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Rider_Id_396,2946,2298,14.0,1159
1,Rider_Id_479,360,951,13.5,176


In [7]:
xdata = dataset.drop(columns=['Time from Pickup to Arrival', 'User Id', 'Order No', 'Arrival at Destination - Time']) 
X = xdata.copy() # I want to run this column everytime I want to restart after changes
X = X.merge(riders, how='left', on=['Rider Id'])
X.drop(columns=['Rider Id'], inplace=True)

X.head(2)

,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,...,Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,...,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,1637,1309,13.8,549
1,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,...,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,396,339,13.6,69


In [8]:
y = dataset['Time from Pickup to Arrival'] # The target

We Split

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,...,Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,...,20.400000,NaN,-1.317755,36.830370,-1.300406,36.829741,1637,1309,13.8,549
1,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,...,26.400000,NaN,-1.351453,36.899315,-1.295004,36.814358,396,339,13.6,69
2,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,...,23.258889,NaN,-1.308284,36.843419,-1.300921,36.828195,1023,242,12.5,114
3,Bike,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,...,19.200000,NaN,-1.281301,36.832396,-1.257147,36.795063,886,283,14.5,113
4,Bike,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,...,15.400000,NaN,-1.266597,36.792118,-1.295041,36.809817,2311,872,14.1,533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21196,Bike,3,Personal,20,3,3:54:38 PM,20,3,3:55:09 PM,20,...,28.600000,NaN,-1.258414,36.804800,-1.275285,36.802702,1270,367,14.4,131
21197,Bike,3,Business,13,6,10:13:34 AM,13,6,10:13:41 AM,13,...,26.000000,NaN,-1.307143,36.825009,-1.331619,36.847976,1023,242,12.5,114
21198,Bike,3,Business,7,4,5:06:16 PM,7,4,5:07:09 PM,7,...,29.200000,NaN,-1.286018,36.897534,-1.258414,36.804800,314,473,13.8,29
21199,Bike,1,Personal,4,3,9:31:39 AM,4,3,9:31:53 AM,4,...,15.000000,NaN,-1.250030,36.874167,-1.279209,36.794872,2451,1821,14.0,1101


Now things are about to get ugly

In [11]:
### START FUNCTION
def cleaner(input_df, nullthreshold=0.9, correlation_thresh=0.95, day_of_month_cols=[], day_of_week_cols=[]):
    input_dfc = input_df.copy()
    

    #########################################################################################
    # The Code below drops columns that have null values exceeding threshold and Columns that have ONLY one value
    for col in input_df.columns:
        if (sum(input_df[col].isnull())/len(input_df[col]) > nullthreshold) or (len(input_df[col].unique()) == 1):
            input_dfc.drop(columns=[col], inplace=True) 
            
    #########################################################################################

    #########################################################################################
    #This code converts time given by am and pm into seconds then applies cosine and sine
    def time_to_seconds(input_df):
        input_dfc = input_df.copy()

        from datetime import datetime

        for time_col in [col for col in input_df.columns if 'Time' in [col[-4:]]]:

            input_dfc[time_col + '_sin(seconds)'] = \
            input_df[time_col].apply(lambda time: np.sin(
                (datetime.strptime(time, '%I:%M:%S %p') - datetime(1900,1,1)).total_seconds() \
                * (2.*np.pi/86400) )) # there are 86400 seconds in a day

            input_dfc[time_col + '_cos(seconds)'] = \
            input_df[time_col].apply(lambda time: np.cos(
                (datetime.strptime(time, '%I:%M:%S %p') - datetime(1900,1,1)).total_seconds() \
                * (2.*np.pi/86400) ))

            input_dfc.drop(columns=[time_col], inplace=True)

        return input_dfc

    input_dfc2 = time_to_seconds(input_dfc)
    #########################################################################################
    
    #########################################################################################
    # This code encodes ['Platform Type', 'Personal or Business']
    
    def one_encoder(input_df, columns):
    
        return pd.get_dummies(input_df, drop_first=True, columns=columns, dtype=float)

    
    input_dfc2 = one_encoder(input_dfc2, ['Platform Type', 'Personal or Business'])
    #########################################################################################
    
    #########################################################################################
    # This code will remove one of a pair of variables that are 95% correlated
    def correlation_drop(df, thresh):
        while True:
            corr_matrix = df.corr(method = "spearman").abs()
            upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(np.bool))
            to_drop = [column for column in upper.columns if any(upper[column] > thresh)]
            if len(to_drop) == 0:
                break
            else:
                df = df.drop(to_drop, axis = 1)
            
        return df
    
    input_dfc2 = correlation_drop(input_dfc2, correlation_thresh)
    #########################################################################################

    print(f"Total of {len([x for x in input_df.columns if x not in input_dfc2])} original columns dropped \n")  
    print(f"Total of {len([x for x in input_dfc2.columns if x not in input_df])} new CLEAN columns formed \n")
    print(f"Dataframe now has {len(input_dfc2.columns)} from {len(input_df.columns)} input columns")
    #print([x for x in input_df.columns if x not in input_dfc2])

    return input_dfc2

### END FUNCTION

In [69]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.neural_network import MLPRegressor
# from sklearn.pipeline import Pipeline

# # Construct the pipeline with a standard scaler and a small neural network
# estimators = []
# estimators.append(('standardize', StandardScaler()))
# estimators.append(('nn', MLPRegressor(hidden_layer_sizes=(5,), max_iter=1000)))
# model = Pipeline(estimators)

# # To begin, let's use only these two features to predict 'cnt' (bicycle count)
# features = ['mnth','hr']
# X = df[features].values
# y = df.cnt

# # We'll use 5-fold cross validation. That is, a random 80% of the data will be used
# # to train the model, and the prediction score will be computed on the remaining 20%.
# # This process is repeated five times such that the training sets in each "fold"
# # are mutually orthogonal.
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score
# kfold = KFold(n_splits=5)

# results = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_squared_error')
# print 'CV Scoring Result: mean=',np.mean(results),'std=',np.std(results)

In [70]:
X_train.head()

,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,...,Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,No_Of_Orders,Age,Average_Rating,No_of_Ratings
6974,Bike,3,Business,15,5,1:15:19 PM,15,5,1:16:19 PM,15,...,28.1,NaN,-1.272639,36.794723,-1.207763,36.882515,2069,2103,14.0,649
1367,Bike,3,Business,3,3,3:45:33 PM,3,3,3:47:25 PM,3,...,28.2,NaN,-1.273647,36.816404,-1.255189,36.782203,173,253,13.5,16
11508,Bike,3,Business,8,4,4:56:14 PM,8,4,4:56:26 PM,8,...,25.6,NaN,-1.301446,36.766138,-1.229720,36.881951,1599,1112,12.3,339
13616,Bike,3,Business,27,2,10:15:02 AM,27,2,10:23:19 AM,27,...,23.7,NaN,-1.285767,36.787561,-1.257147,36.795063,3737,1884,14.2,1054
7251,Bike,3,Business,10,1,1:11:29 PM,10,1,1:12:20 PM,10,...,18.3,NaN,-1.324488,36.897792,-1.307357,36.820216,282,353,14.0,23


 Actually for the first model, I will drop the 'Arrival at Destination - Time' column, I believe it is directly linked to the predicted y
 As y is the difference between the 'Arrival at Destination - Time' and 'Pick up time'

Now we apply our ultimate cleaner

In [12]:
day_of_month_cols = [x for x in X.columns if x[-5:] == 'Month']
day_of_week_cols = [x for x in X.columns if x[-(len('(Mo = 1)')):] == '(Mo = 1)']
X_train = cleaner(X_train, day_of_month_cols=day_of_month_cols, day_of_week_cols=day_of_week_cols)


Total of 16 original columns dropped 

Total of 6 new CLEAN columns formed 

Dataframe now has 18 from 28 input columns


In [13]:
X_test = cleaner(X_test, day_of_month_cols=day_of_month_cols, day_of_week_cols=day_of_week_cols)

Total of 16 original columns dropped 

Total of 6 new CLEAN columns formed 

Dataframe now has 18 from 28 input columns


In [112]:
# corr_matrix = X_train.corr(method = "spearman").abs()
# sns.set(font_scale = 1.0)
# f, ax = plt.subplots(figsize=(11, 9))
# sns.heatmap(corr_matrix, cmap= "YlGnBu", square=True, ax = ax)
# f.tight_layout()
# plt.savefig("correlation_matrix.png", dpi = 1080)


In [74]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
to_drop

[]

In [75]:
# X_train = X_train.drop(columns='No_of_Ratings') # Number of ratings has no real 
# X_test = X_test.drop(columns='No_of_Ratings')

Our data is ready

We will standardize the columns which have a max absolute value that is greater than 1

In [77]:
cols_to_standard = [x for x in X_train.columns if max(X_train[x].apply(abs)) > 1]

In [ ]:
X1 = X['Temperature']

In [26]:
scaled_features_df['Temperature'] = scaled_features_df['Temperature'].fillna(scaled_features_df['Temperature'].median())

In [21]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression# Sequential Forward Selection(sfs)
sfs = SFS(LinearRegression(),
           k_features=11,
           forward=True,
           floating=False,
           scoring = 'r2',
           cv = 0)

In [14]:
#X  = cleaner(X, day_of_month_cols=day_of_month_cols, day_of_week_cols=day_of_week_cols)

In [18]:
from sklearn_pandas import DataFrameMapper

mapper = DataFrameMapper([(X_train.columns, StandardScaler())])
scaled_features = mapper.fit_transform(X_train.copy(), 4)
scaled_features_df = pd.DataFrame(scaled_features, index=X_train.index, columns=X_train.columns)

In [19]:
scaled_features_df

,Placement - Day of Month,Placement - Weekday (Mo = 1),Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Placement - Time_sin(seconds),Placement - Time_cos(seconds),Platform Type_2,Platform Type_3,Platform Type_4,Personal or Business_Personal
13751,1.287564,-0.789024,-0.089711,0.173983,0.795785,-0.431381,1.385754,0.813318,-0.285609,-0.556138,0.125050,-0.581826,0.365211,-0.887368,-0.22162,0.418918,-0.027697,-0.469943
12246,1.401329,1.124127,-0.622637,0.589180,0.731400,-0.284248,0.345291,0.597710,0.491479,-0.199173,-0.098444,0.028425,-0.942985,0.219599,-0.22162,0.418918,-0.027697,2.127917
19329,-0.987737,-1.426741,0.976141,0.367742,0.662568,0.886478,2.596884,2.772111,-0.163713,0.197969,0.236797,-0.208758,0.921697,-0.654804,-0.22162,0.418918,-0.027697,2.127917
115,0.491209,0.486410,-0.089711,0.367742,-0.987129,-1.819065,-0.440054,-0.248500,-0.175776,0.823816,-0.433685,0.438553,0.726864,-0.782140,-0.22162,-2.387103,-0.027697,2.127917
5240,-0.191382,1.124127,0.976141,0.146303,0.860269,-0.774549,-1.416143,1.301561,-0.709707,-0.803386,-1.774649,-0.742419,-0.926839,0.186042,-0.22162,0.418918,-0.027697,-0.469943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7725,-0.646442,-1.426741,1.686709,-1.403765,-1.657217,0.980599,-1.030896,-2.407849,-1.043652,-1.123263,1.242520,-0.831362,-0.780090,-0.085006,-0.22162,0.418918,-0.027697,-0.469943
13980,-1.442797,-0.789024,-1.333205,-1.265366,0.216806,0.100669,0.482127,-0.129857,-0.893821,-0.298073,0.013303,-0.700418,-1.438642,2.022797,-0.22162,-2.387103,-0.027697,2.127917
5860,-0.077616,0.486410,-0.444995,1.198136,0.795785,-0.431381,0.178090,-0.992154,0.311809,-0.148178,0.236797,0.127251,-0.915124,0.162199,-0.22162,0.418918,-0.027697,-0.469943
7143,-0.305147,-0.151307,-0.444995,-0.933209,-0.455130,0.451124,0.157676,-0.426035,-0.938897,0.044984,0.125050,-0.655946,0.331570,-0.888926,-0.22162,0.418918,-0.027697,2.127917


In [27]:
sfs.fit(scaled_features_df, y_train)
sfs.k_feature_names_ 

('Placement - Day of Month',
 'Distance (KM)',
 'Pickup Lat',
 'Pickup Long',
 'Destination Lat',
 'No_Of_Orders',
 'Average_Rating',
 'No_of_Ratings',
 'Placement - Time_sin(seconds)',
 'Placement - Time_cos(seconds)',
 'Personal or Business_Personal')

In [32]:

sfs.scorer(LinearRegression(), scaled_features_df, y_train)

NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer

In [83]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

In [84]:
models = [LinearRegression()]
polinomial_features = PolynomialFeatures(2)


for model in models:
    numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
    
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, cols_to_standard)])
    
    steps = [('imputer', SimpleImputer(strategy='mean')), ('polf', polinomial_features), 
          ('Scaler', StandardScaler()),
          ('model', model)]
    
    pipe = Pipeline(steps)
    pipe.fit(X_train, y_train)
    print(model)
    print("model score: %.3f" % pipe.score(X_test, y_test))
    print('\n')
    

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
model score: 0.378




In [85]:
len(X_train['Age'])

16960

In [86]:
r2_score(y_test, y_pred)

NameError: name 'r2_score' is not defined

In [87]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[('impute', SimpleImputer()),
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

NameError: name 'numeric_features' is not defined

In [88]:
riders.head()

,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Rider_Id_396,2946,2298,14.0,1159
1,Rider_Id_479,360,951,13.5,176
2,Rider_Id_648,1746,821,14.3,466
3,Rider_Id_753,314,980,12.5,75
4,Rider_Id_335,536,1113,13.7,156


In [89]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21201 entries, 0 to 21200
Data columns (total 28 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Vehicle Type                               21201 non-null  object 
 1   Platform Type                              21201 non-null  int64  
 2   Personal or Business                       21201 non-null  object 
 3   Placement - Day of Month                   21201 non-null  int64  
 4   Placement - Weekday (Mo = 1)               21201 non-null  int64  
 5   Placement - Time                           21201 non-null  object 
 6   Confirmation - Day of Month                21201 non-null  int64  
 7   Confirmation - Weekday (Mo = 1)            21201 non-null  int64  
 8   Confirmation - Time                        21201 non-null  object 
 9   Arrival at Pickup - Day of Month           21201 non-null  int64  
 10  Arrival at Pickup - We

In [90]:
sample = pd.read_csv('SampleSubmission.csv')

In [91]:
sample

,Order_No,Time from Pickup to Arrival
0,Order_No_19248,567.0
1,Order_No_12736,4903.0
2,Order_No_768,5649.0
3,Order_No_15332,NaN
4,Order_No_21373,NaN
...,...,...
7063,Order_No_3612,NaN
7064,Order_No_7657,NaN
7065,Order_No_1969,NaN
7066,Order_No_10591,NaN


In [92]:
finaltest = pd.read_csv("Test.csv")

In [93]:
finaltest.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Pickup - Weekday (Mo = 1),Pickup - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,3,4:44:10 PM,27,3,...,3,5:06:47 PM,8,NaN,NaN,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192
1,Order_No_12736,User_Id_3647,Bike,3,Business,17,5,12:57:35 PM,17,5,...,5,1:25:37 PM,5,NaN,NaN,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868
2,Order_No_768,User_Id_2154,Bike,3,Business,27,4,11:08:14 AM,27,4,...,4,11:57:54 AM,5,22.8,NaN,-1.290894,36.822971,-1.276574,36.851365,Rider_Id_26
3,Order_No_15332,User_Id_2910,Bike,3,Business,17,1,1:51:35 PM,17,1,...,1,2:16:52 PM,5,24.5,NaN,-1.290503,36.809646,-1.303382,36.790658,Rider_Id_685
4,Order_No_21373,User_Id_1205,Bike,3,Business,11,2,11:30:28 AM,11,2,...,2,11:56:04 AM,6,24.4,NaN,-1.281081,36.814423,-1.266467,36.792161,Rider_Id_858


In [94]:
finaltest.columns

Index(['Order No', 'User Id', 'Vehicle Type', 'Platform Type',
       'Personal or Business', 'Placement - Day of Month',
       'Placement - Weekday (Mo = 1)', 'Placement - Time',
       'Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
       'Confirmation - Time', 'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Arrival at Pickup - Time',
       'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)', 'Pickup - Time',
       'Distance (KM)', 'Temperature', 'Precipitation in millimeters',
       'Pickup Lat', 'Pickup Long', 'Destination Lat', 'Destination Long',
       'Rider Id'],
      dtype='object')

In [95]:
len(dataset['Order No'].unique())

21201

In [96]:
finaltest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7068 entries, 0 to 7067
Data columns (total 25 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Order No                              7068 non-null   object 
 1   User Id                               7068 non-null   object 
 2   Vehicle Type                          7068 non-null   object 
 3   Platform Type                         7068 non-null   int64  
 4   Personal or Business                  7068 non-null   object 
 5   Placement - Day of Month              7068 non-null   int64  
 6   Placement - Weekday (Mo = 1)          7068 non-null   int64  
 7   Placement - Time                      7068 non-null   object 
 8   Confirmation - Day of Month           7068 non-null   int64  
 9   Confirmation - Weekday (Mo = 1)       7068 non-null   int64  
 10  Confirmation - Time                   7068 non-null   object 
 11  Arrival at Pickup